In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import plotly.express as px


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Replace 'path/to/your/excel_file.xlsx' with the actual path to your file in Google Drive
Innovative_BU_erbitux_sales_2023_to_2025_HA_path = '/content/drive/MyDrive/Merck/Innovative_BU_erbitux_sales_2023_to_2025_HA.xlsx'
# Replace 'Sheet1' with the name or index of the sheet you want to read
Innovative_BU_erbitux_sales_2023_to_2025_HA = pd.read_excel(Innovative_BU_erbitux_sales_2023_to_2025_HA_path)

display(Innovative_BU_erbitux_sales_2023_to_2025_HA)

In [ ]:
# ['PRINCESS MARGARET HOSP', 'UNITED CHRISTIAN HOSPITAL', 'TSEUNG KWAN O HOSPITAL', 'QUEEN ELIZABETH HOSPITAL', 'PRINCE OF WALES HOSPITAL', 'QUEEN MARY HOSPITAL C/O PHARMACY', 'PAMELA YOUDE NETHERSOLE EASTERN HOSPITAL', 'TUEN MUN HOSPITAL']
HA_hospital_list = ['UNITED CHRISTIAN HOSPITAL', 'TSEUNG KWAN O HOSPITAL', 'QUEEN ELIZABETH HOSPITAL']

# top_sales_hospital = ['QUEEN ELIZABETH HOSPITAL']
HA_hospital_table_list = []


# # Filter the DataFrame based on 'Brand Detail' being in the list and 'AmountInHKD' being greater than 0

for HA_hospital in HA_hospital_list:
  table_name = f'Innovative_BU_erbitux_sales_2023_to_2025_HA_{HA_hospital}'
  table_name = Innovative_BU_erbitux_sales_2023_to_2025_HA[
      (Innovative_BU_erbitux_sales_2023_to_2025_HA['SoldToCustomerName'] == HA_hospital) &
      # (Innovative_BU_erbitux_sales_2023_to_2025_HA['ShipToCode From ImportData'] == '70145698') &
      (Innovative_BU_erbitux_sales_2023_to_2025_HA['Brand Detail'] == 'ERBITUX 5MG/ML INJ 20ML 1\'S')
  ]
  # Innovative_BU_erbitux_sales_2023_to_2025_HA_filtered = Innovative_BU_erbitux_sales_2023_to_2025_HA_filtered.drop('Unnamed: 0', axis=1)

  # # Display the filtered DataFrame
  # display(table_name.head())

  print(f"The number of rows in the Innovative_BU_erbitux_sales_2023_to_2025_HA_{HA_hospital} is: {table_name.shape[0]}")
  HA_hospital_table_list.append(table_name)

In [ ]:
# write a for loop to iterate via each of the table in the HA_hospital_table_list to filter out colum "SoldToCustomerName", "Brand Detail", "Std Counting Unit", "InvoiceDate", "SaleOrderNo", "Counting Unit", "AmountInHKD", "YearQuarter" and "Time_Between_Invoices"

# Columns to filter out
columns_to_filter = ['Year', 'Quarter', 'Month',  'Original BU', 'ShipToCustomerName',
                   'SoldToCode From ImportData', 'ShipToCode From ImportData', 'QTY',
                   'Std/ Bonus...',  'Sales Rep', 'Sub Channel', 'LocationDesc']

# Create a new list to store the dataframes with filtered columns
HA_hospital_table_list_filtered = []

# Iterate through each table in the HA_hospital_table_list
for table in HA_hospital_table_list:
  # Drop the specified columns from the current table
  table_filtered = table.drop(columns=columns_to_filter, errors='ignore')  # Use errors='ignore' to avoid errors if a column doesn't exist
  # Append the filtered table to the new list
  HA_hospital_table_list_filtered.append(table_filtered)

# Display the first few rows of the first filtered table as an example
display(HA_hospital_table_list_filtered[0])

In [ ]:
# prompt: from HA_hospital_table_list[2], the columns SaleOrderNo consist of duplicates, merge the row with the same SaleOrderNo together. their Counting Unit, AmountInHKD, Std Counting Unit and  Time_Between_Invoices should be sum up tgt. the ended table should be short by InvoiceDate

HA_hospital_table_list_merged_same_order = []

for df_hospital in HA_hospital_table_list_filtered:
  # Group by 'SaleOrderNo' and sum the specified columns
  df_merged = df_hospital.groupby(['SaleOrderNo', 'InvoiceDate', 'SoldToCustomerName', 'Brand Detail']).agg(
      {'Counting Unit': 'sum', 'AmountInHKD': 'sum', 'Std Counting Unit': 'sum', 'Time_Between_Invoices': 'sum'}
  ).reset_index()

  # Sort by 'InvoiceDate'
  df_merged = df_merged.sort_values(by='InvoiceDate')

  HA_hospital_table_list_merged_same_order.append(df_merged)


display(HA_hospital_table_list_merged_same_order[0])
print(f"The number of rows in the merged and sorted DataFrame for {HA_hospital_list[0]} is: {HA_hospital_table_list_merged_same_order[0].shape[0]}")




In [ ]:
# Visualize the time interval between each invoice date and AmountInHKD
# skip the first row as its time_between_invoices is always 0

fig = px.scatter(HA_hospital_table_list_merged_same_order[2].iloc[1:].dropna(),
                 x='InvoiceDate',
                 y='Time_Between_Invoices',
                #  color='AmountInHKD',
                 size='AmountInHKD',
                 title='Time Between Invoices vs. Invoice Date colored by Amount in HKD',
                 labels={'InvoiceDate': 'Invoice Date', 'Time_Between_Invoices': 'Time Between Invoices (Days)', 'AmountInHKD': 'Amount in HKD'})

fig.update_traces(mode='lines+markers') # Add lines connecting the markers

# Update layout for x-axis to show ticks every 1 month
fig.update_layout(
    xaxis=dict(
        # Removed tickmode='auto'
        dtick='M2',  # Set x-axis ticks every 1 month
        title='Invoice Date'
    ),
    yaxis_title="Time Between Invoices (Days)"
)

fig.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform
import numpy as np

def enhanced_RF(sales_table):
    # Assuming sales_table is already created and has 'InvoiceDate', 'Time_Between_Invoices', and 'Counting Unit'
    # Drop rows with NaN in 'Time_Between_Invoices' as the first row will have NaN
    hospital_name = sales_table['SoldToCustomerName'].iloc[0]
    print(f"Processing: {hospital_name}")
    df_model_advanced = sales_table.dropna(subset=['Time_Between_Invoices']).copy()

    # Convert InvoiceDate to a numerical representation (days since the earliest date in the dataset)
    # We still calculate this for potential future use or visualization, but won't predict it directly
    if 'earliest_date' not in locals():
        earliest_date = df_model_advanced['InvoiceDate'].min()
    df_model_advanced['Days_Since_Earliest'] = (df_model_advanced['InvoiceDate'] - earliest_date).dt.days


    # Advanced Feature Engineering

    # Lagged features (previous 1, 2, and 3 invoices)
    for i in range(1, 4):
        df_model_advanced[f'Lag_{i}_Time_Between_Invoices'] = df_model_advanced['Time_Between_Invoices'].shift(i)
        df_model_advanced[f'Lag_{i}_Counting_Unit'] = df_model_advanced['Counting Unit'].shift(i)

    # Rolling mean features (window of 3 invoices)
    window_size = 3
    df_model_advanced['Rolling_Mean_Time_Between_Invoices'] = df_model_advanced['Time_Between_Invoices'].rolling(window=window_size).mean()
    df_model_advanced['Rolling_Mean_Counting_Unit'] = df_model_advanced['Counting Unit'].rolling(window=window_size).mean()


    # Drop rows with NaN created by shifting and rolling
    df_model_advanced.dropna(inplace=True)

    # Select features (X) and target (y)
    # The target is now 'Time_Between_Invoices'
    X_advanced = df_model_advanced[['Lag_1_Time_Between_Invoices', 'Lag_2_Time_Between_Invoices', 'Lag_3_Time_Between_Invoices',
                                   'Lag_1_Counting_Unit', 'Lag_2_Counting_Unit', 'Lag_3_Counting_Unit',
                                   'Rolling_Mean_Time_Between_Invoices', 'Rolling_Mean_Counting_Unit',
                                   'Counting Unit']] # Include the current Counting Unit as a feature
    y_advanced = df_model_advanced['Time_Between_Invoices'] # Target is Time_Between_Invoices


    # Time-based split
    # We still need a train/test split for final evaluation after tuning
    split_index_advanced = int(len(df_model_advanced) * 0.83)
    X_train_advanced, X_test_advanced = X_advanced[:split_index_advanced], X_advanced[split_index_advanced:]
    y_train_advanced, y_test_advanced = y_advanced[:split_index_advanced], y_advanced[split_index_advanced:]

    print("Training set shapes for predicting Time_Between_Invoices:")
    print("X_train_advanced:", X_train_advanced.shape)
    print("y_train_advanced:", y_train_advanced.shape)
    print("\nTesting set shapes for predicting Time_Between_Invoices:")
    print("X_test_advanced:", X_test_advanced.shape)
    y_test_advanced = y_test_advanced.reset_index(drop=True) # Reset index to align with new df
    print("y_test_advanced:", y_test_advanced.shape)


    # Define the parameter distribution for Randomized Search for Random Forest
    param_dist_rf = {
        'n_estimators': randint(100, 1000),
        'max_features': ['sqrt', 'log2'], # Features to consider at each split (removed 'auto' as it's deprecated)
        'max_depth': randint(10, 110),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'bootstrap': [True, False]
    }

    # Initialize the Random Forest Regressor
    rf_tuned = RandomForestRegressor(random_state=42)

    # Initialize TimeSeriesSplit for cross-validation
    # n_splits determines the number of splits. Increase for more robust evaluation.
    tscv = TimeSeriesSplit(n_splits=5) # Using TimeSeriesSplit

    # Initialize RandomizedSearchCV
    random_search_rf = RandomizedSearchCV(
        rf_tuned,
        param_distributions=param_dist_rf,
        n_iter=100,  # Number of different combinations to try (can adjust)
        cv=tscv,     # Use TimeSeriesSplit for cross-validation
        scoring='neg_mean_squared_error', # Use negative MSE for scoring
        random_state=42,
        n_jobs=-1    # Use all available cores
    )

    # Perform randomized search on the training data
    # We fit on the training data because TimeSeriesSplit will handle the temporal splits within the training set
    random_search_rf.fit(X_train_advanced, y_train_advanced)

    # Get the best parameters and the best model
    best_params_rf = random_search_rf.best_params_
    # best_params_rf = {'bootstrap': False, 'max_depth': 105, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 848}
    best_rf_model_tuned = random_search_rf.best_estimator_
    # best_rf_model_tuned = RandomForestRegressor(bootstrap=False, max_depth=105, max_features='log2', min_samples_leaf=5, min_samples_split=8, n_estimators=848, random_state=42)

    print("\nBest hyperparameters found for tuned Random Forest:", best_params_rf)
    print("\nBest hyperparameters found for best_rf_model_tuned:", best_rf_model_tuned)
    # Best hyperparameters found for tuned Random Forest: {'bootstrap': False, 'max_depth': 105, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 848}
    # RandomForestRegressor(bootstrap=False, max_depth=105, max_features='log2', min_samples_leaf=5, min_samples_split=8, n_estimators=848, random_state=42)


    # Make predictions on the test set using the best tuned model
    y_pred_time_interval_rf_tuned = best_rf_model_tuned.predict(X_test_advanced)

    # Evaluate the best tuned Random Forest model (predicting Time_Between_Invoices)
    mse_time_interval_rf_tuned = mean_squared_error(y_test_advanced, y_pred_time_interval_rf_tuned)
    rmse_time_interval_rf_tuned = np.sqrt(mse_time_interval_rf_tuned)
    r2_time_interval_rf_tuned = r2_score(y_test_advanced, y_pred_time_interval_rf_tuned)

    print(f"\nTuned Random Forest Model predicting Time_Between_Invoices:")
    print(f"Mean Squared Error: {mse_time_interval_rf_tuned:.2f}")
    print(f"Root Mean Squared Error: {rmse_time_interval_rf_tuned:.2f}")
    print(f"R-squared: {r2_time_interval_rf_tuned:.2f}")

    # Display predictions vs actual time intervals for inspection
    results_df_time_interval_rf_tuned = pd.DataFrame({'Actual_Time_Between': y_test_advanced, 'Predicted_Time_Between': y_pred_time_interval_rf_tuned})
    # Add the corresponding InvoiceDate for visualization
    results_df_time_interval_rf_tuned['InvoiceDate'] = df_model_advanced.iloc[split_index_advanced:]['InvoiceDate'].values
    display(results_df_time_interval_rf_tuned.head())

    # Visualize the actual vs predicted time intervals from tuned Random Forest
    # fig_rf_tuned_interval = px.line(results_df_time_interval_rf_tuned, x='InvoiceDate', y=['Actual_Time_Between', 'Predicted_Time_Between'],
    #               title='Actual vs Predicted Time Between Invoices (Tuned Random Forest)',
    #               labels={'InvoiceDate': 'Invoice Date', 'value': 'Time Between Invoices (Days)'})

    # fig_rf_tuned_interval.update_layout(
    #     xaxis=dict(
    #         dtick='M2',  # Set x-axis ticks every 2 months
    #         title='Invoice Date'
    #     ),
    #     yaxis_title="Time Between Invoices (Days)"
    # )
    # fig_rf_tuned_interval.show()


    # To forecast the next invoice date:
    # We need to predict the time interval for the next invoice.
    # The features for the next prediction will be based on the last row of the df_model_advanced DataFrame.

    last_row_advanced = df_model_advanced.iloc[-1]

    # Prepare features for the next prediction (predicting the time interval)
    # Features are: 'Lag_1_Time_Between_Invoices', 'Lag_2_Time_Between_Invoices', 'Lag_3_Time_Between_Invoices',
    # 'Lag_1_Counting_Unit', 'Lag_2_Counting_Unit', 'Lag_3_Counting_Unit',
    # 'Rolling_Mean_Time_Between_Invoices', 'Rolling_Mean_Counting_Unit', 'Counting Unit'

    # For the next prediction:
    # Lag_1_Time_Between_Invoices will be the last actual Time_Between_Invoices
    # Lag_2_Time_Between_Invoices will be the last actual Lag_1_Time_Between_Invoices
    # Lag_3_Time_Between_Invoices will be the last actual Lag_2_Time_Between_Invoices
    # Similarly for Counting_Unit lagged features

    # Rolling means will be recalculated including the last actual Time_Between_Invoices and Counting Unit
    # The next 'Counting Unit' (the last feature) needs to be estimated. We'll use the average of the last few actual counting units.

    last_actual_time_between = last_row_advanced['Time_Between_Invoices']
    last_lag1_time_between = last_row_advanced['Lag_1_Time_Between_Invoices']
    last_lag2_time_between = last_row_advanced['Lag_2_Time_Between_Invoices'] # This will be the Lag_3 for the next step

    last_actual_counting_unit = last_row_advanced['Counting Unit']
    last_lag1_counting_unit = last_row_advanced['Lag_1_Counting_Unit']
    last_lag2_counting_unit = last_row_advanced['Lag_2_Counting_Unit'] # This will be the Lag_3 for the next step

    # Recalculate rolling means including the last actual data
    last_rolling_mean_time_between = df_model_advanced['Time_Between_Invoices'].tail(window_size).mean()
    last_rolling_mean_counting_unit = df_model_advanced['Counting Unit'].tail(window_size).mean()

    # Estimate the next Counting Unit (using the mean of the last window_size)
    next_counting_unit_estimated = df_model_advanced['Counting Unit'].tail(window_size).mean()


    next_features_time_interval = np.array([[
        last_actual_time_between,        # Lag_1_Time_Between_Invoices for next step
        last_lag1_time_between,          # Lag_2_Time_Between_Invoices for next step
        last_lag2_time_between,          # Lag_3_Time_Between_Invoices for next step
        last_actual_counting_unit,       # Lag_1_Counting_Unit for next step
        last_lag1_counting_unit,         # Lag_2_Counting_Unit for next step
        last_lag2_counting_unit,         # Lag_3_Counting_Unit for next step
        last_rolling_mean_time_between,  # Rolling_Mean_Time_Between_Invoices for next step
        last_rolling_mean_counting_unit, # Rolling_Mean_Counting_Unit for next step
        next_counting_unit_estimated     # Estimated Counting Unit for the next invoice
    ]])


    predicted_next_time_interval_rf_tuned = best_rf_model_tuned.predict(next_features_time_interval)[0]

    # Ensure the predicted time interval is not negative
    predicted_next_time_interval_rf_tuned = max(0, predicted_next_time_interval_rf_tuned)


    # Get the last actual invoice date
    last_actual_invoice_date = df_model_advanced['InvoiceDate'].iloc[-1]

    # Forecast the next invoice date by adding the predicted time interval to the last actual invoice date
    forecasted_next_invoice_date_rf_tuned = last_actual_invoice_date + pd.to_timedelta(predicted_next_time_interval_rf_tuned, unit='D')

    print(f"\nLast Actual Invoice Date: {last_actual_invoice_date.strftime('%Y-%m-%d')}")
    print(f"Predicted Time Interval for Next Invoice: {predicted_next_time_interval_rf_tuned:.2f} days")
    print(f"Forecasted Next Invoice Date (using Tuned Random Forest model): {forecasted_next_invoice_date_rf_tuned.strftime('%Y-%m-%d')}")

    # Visualize the actual vs predicted time intervals and the forecasted date
    results_melted_forecast = results_df_time_interval_rf_tuned.melt(id_vars='InvoiceDate', var_name='Type', value_name='Time_Between_Invoices')

    # Add the forecast point to the melted dataframe
    forecast_data = pd.DataFrame({
        'InvoiceDate': [forecasted_next_invoice_date_rf_tuned],
        'Time_Between_Invoices': [predicted_next_time_interval_rf_tuned],
        'Type': ['Forecasted Next Invoice Date (RF)']
    })

    results_melted_forecast = pd.concat([results_melted_forecast, forecast_data], ignore_index=True)

    # Add an 'Invoice_Order' column for the x-axis
    # We need to re-calculate this based on the combined dataframe for correct sequential numbering
    results_melted_forecast = results_melted_forecast.sort_values(by='InvoiceDate').reset_index(drop=True)
    results_melted_forecast['Invoice_Order'] = results_melted_forecast.index + 1


    fig_forecast = px.line(results_melted_forecast,
                          x='Invoice_Order',
                          y='Time_Between_Invoices',
                          color='Type',
                          title='Actual vs Predicted Time Between Invoices and Forecasted Next Date (Tuned RF)',
                          labels={'Invoice_Order': 'Invoice Number', 'Time_Between_Invoices': 'Time Between Invoices (Days)'},
                          custom_data=['InvoiceDate']) # Include InvoiceDate in custom data for hover

    # Customize hover template to show Invoice Date
    fig_forecast.update_traces(hovertemplate="Invoice Number: %{x}<br>Time Between Invoices: %{y:.2f} days<br>Invoice Date: %{customdata[0]|%Y-%m-%d}<extra></extra>")


    # Add markers for all points, especially the forecast point
    fig_forecast.update_traces(mode='lines+markers', selector=dict(type='scatter', mode='lines+markers'))
    fig_forecast.update_traces(mode='markers', selector=dict(type='scatter', name='Forecasted Next Invoice Date (RF)')) # Ensure forecast has a marker


    fig_forecast.update_layout(xaxis_title="Invoice Number", yaxis_title="Time Between Invoices (Days)")


    fig_forecast.show()
    ############################################################################################################################################

    fig_forecast_2 = px.line(results_melted_forecast, x='InvoiceDate', y='Time_Between_Invoices', color='Type',
                          title=f"Actual vs Predicted Time Between Invoices and Forecasted Next Date (Tuned RF) for {hospital_name}",
                          labels={'InvoiceDate': 'Invoice Date', 'Time_Between_Invoices': 'Time Between Invoices (Days)'})

    # Add markers for all points, especially the forecast point
    fig_forecast_2.update_traces(mode='lines+markers', selector=dict(type='scatter', mode='lines+markers'))
    fig_forecast_2.update_traces(mode='markers', selector=dict(type='scatter', name='Forecasted Next Invoice Date (RF)')) # Ensure forecast has a marker

    fig_forecast_2.update_layout(
        xaxis=dict(
            dtick='M1',  # Set x-axis ticks every 2 months
            title='Invoice Date'
        ),
        yaxis_title="Time Between Invoices (Days)"
    )
    fig_forecast_2.show()


In [ ]:
for hospital_table in HA_hospital_table_list_merged_same_order[:1]:
  display(hospital_table.iloc[1:].head(3))
  results_melted_forecast_2 = enhanced_RF(hospital_table.iloc[1:]) # drop the first row as it is always 0 in time intervals
  print("***********************************************************************************************************************************************")
  print("***********************************************************************************************************************************************")
  print("====================================================================================================================================================================== \n")
  print("====================================================================================================================================================================== \n")
  print("====================================================================================================================================================================== \n")
  print("====================================================================================================================================================================== \n")

In [ ]:
display(results_melted_forecast_2)

fig_forecast_2 = px.line(results_melted_forecast_2, x='InvoiceDate', y='Time_Between_Invoices', color='Type',
                      title=f"Actual vs Predicted Time Between Invoices and Forecasted Next Date (Tuned RF)",
                      labels={'InvoiceDate': 'Invoice Date', 'Time_Between_Invoices': 'Time Between Invoices (Days)'})

# Add markers for all points, especially the forecast point
fig_forecast_2.update_traces(mode='lines+markers', selector=dict(type='scatter', mode='lines+markers'))
fig_forecast_2.update_traces(mode='markers', selector=dict(type='scatter', name='Forecasted Next Invoice Date (RF)')) # Ensure forecast has a marker

fig_forecast_2.update_layout(
    xaxis=dict(
        dtick='M1',  # Set x-axis ticks every 2 months
        title='Invoice Date'
    ),
    yaxis_title="Time Between Invoices (Days)"
)
fig_forecast_2.show()